# Propositionalization: Occupancy detection

In this notebbok, we compare getML's FastProp against well-known feature engineering libraries featuretools and tsfresh.

Summary:

- Prediction type: __Binary classification__
- Domain: __Energy__
- Prediction target: __Room occupancy__
- Source data: __1 table, 32k rows__
- Population size: __32k__

## Background

A common approach to feature engineering is to generate attribute-value representations from relational data by applying a fixed set of aggregations to columns of interest and perform a feature selection on the (possibly large) set of generated features afterwards. In academia, this approach is called _propositionalization._

getML's [FastProp](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#fastprop) is an implementation of this propositionalization approach that has been optimized for speed and memory efficiency. In this notebook, we want to demonstrate how – well – fast FastProp is. To this end, we will benchmark FastProp against the popular feature engineering libraries [featuretools](https://www.featuretools.com/) and [tsfresh](https://tsfresh.readthedocs.io/en/latest/). Both of these libraries use propositionalization approaches for feature engineering.

Our use case here is a public domain data set for predicting room occupancy from sensor data. For further details about the data set refer to [the full notebook](../occupancy.ipynb).

## Analysis

1. [Loading data](#1.-Loading-data)
2. [Predictive modeling](#2.-Predictive-modeling)
3. [Comparison](#3.-Comparison)

Let's get started with the analysis and set-up your session:

In [1]:
import datetime
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
from pathlib import Path

import sys
import time
from urllib import request

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

%matplotlib inline

import getml

print(f"getML API version: {getml.__version__}\n")

getml.engine.launch(home_directory=Path.home(), allow_remote_ips=True, token='token')
getml.engine.set_project("occupancy")

getML API version: 1.4.0

getML engine is already running.
Loading pipelines... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Loading hyperopts... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

Connected to project 'occupancy'


In [2]:
parent = Path(os.getcwd()).parent.as_posix()

if parent not in sys.path:
   sys.path.append(parent) 

from utils import Benchmark, FTTimeSeriesBuilder, TSFreshBuilder

### 1. Loading data


The data set can be downloaded directly from GitHub. It is conveniently separated into a train, a validation and a testing set. This allows us to directly benchmark our results against the results of the original paper later.

In [3]:
data_test, data_train, data_validate = getml.datasets.load_occupancy(roles=True)


Loading population_train...
 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

Loading population_test...
 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

Loading population_validation...
 100% |██████████| [elapsed: 00:00, remaining: 00:00]          


In [4]:
data_all, split = getml.data.split.concat(
    "data_all",
    train=data_train,
    validation=data_validate,
    test=data_test,
)

The train set looks like this:

In [5]:
data_train

name,date,Occupancy,Temperature,Humidity,Light,CO2,HumidityRatio
role,time_stamp,target,numerical,numerical,numerical,numerical,numerical
unit,time stamp,,,,,,
0,2015-02-11 14:48:00,1,21.76,31.1333,437.3333,1029.6667,0.005021
1,2015-02-11 14:49:00,1,21.79,31,437.3333,1000,0.005009
2,2015-02-11 14:50:00,1,21.7675,31.1225,434,1003.75,0.005022
3,2015-02-11 14:51:00,1,21.7675,31.1225,439,1009.5,0.005022
4,2015-02-11 14:51:59,1,21.79,31.1333,437.3333,1005.6667,0.00503
,...,...,...,...,...,...,...
9747,2015-02-18 09:15:00,1,20.815,27.7175,429.75,1505.25,0.004213
9748,2015-02-18 09:16:00,1,20.865,27.745,423.5,1514.5,0.00423


### 2. Predictive modeling

We loaded the data, defined the roles, units and the abstract data model. Next, we create a getML pipeline for relational learning.

#### 2.1 Propositionalization with getML's FastProp

We use all possible aggregations. Because tsfresh and featuretools are single-threaded, we limit our FastProp algorithm to one thread as well, to ensure a fair comparison.

In [6]:
# Our forecast horizon is 0.
# We do not predict the future, instead we infer
# the present state from current and past sensor data.
horizon = 0.0

# We do not allow the time series features
# to use target values from the past.
# (Otherwise, we would need the horizon to
# be greater than 0.0).
allow_lagged_targets = False

# We want our time series features to only use
# data from the last 15 minutes
memory = getml.data.time.minutes(15)

time_series = getml.data.TimeSeries(
    population=data_all,
    split=split,
    time_stamps="date",
    horizon=horizon,
    memory=memory,
    lagged_targets=allow_lagged_targets,
)

time_series

data model

  population:
    columns:
    - Temperature: numerical
    - Humidity: numerical
    - Light: numerical
    - CO2: numerical
    - HumidityRatio: numerical
    - ...

    joins:
    - right: 'data_all'
      time_stamps: (population.date, data_all.date)
      relationship: 'many-to-many'
      memory: 900.0
      horizon: 0.0
      lagged_targets: False

  data_all:
    columns:
    - Temperature: numerical
    - Humidity: numerical
    - Light: numerical
    - CO2: numerical
    - HumidityRatio: numerical
    - ...


container

  population
      subset       name       rows   type
  0   test         data_all   8142   View
  1   train        data_all   9753   View
  2   validation   data_all   2665   View

  peripheral
      name        rows   type     
  0   data_all   20560   DataFrame

In [7]:
feature_learner = getml.feature_learning.FastProp(
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
    aggregation=getml.feature_learning.FastProp.agg_sets.All,
    num_threads=1,
)

Next, we create the pipeline. In contrast to our usual approach, we create _two pipelines_ in
this notebook. One for feature learning (suffix `_fl`) and one for predicition (suffix `_pr`).
This allows for a fair comparison of runtimes.

In [8]:
pipe_fp_fl = getml.pipeline.Pipeline(
    feature_learners=[feature_learner],
    data_model=time_series.data_model,
    tags=["feature learning", "fastprop"],
)

In [9]:
pipe_fp_fl.check(time_series.train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.


The wrappers around featuretools and tsfresh fit on the training set and then return the training features. We therefore measure the time it takes getML's FastProp algorithm to fit on the training set and create the training features.

In [10]:
benchmark = Benchmark()

In [11]:
with benchmark("fastprop"):
    pipe_fp_fl.fit(time_series.train)
    fastprop_train = pipe_fp_fl.transform(time_series.train, df_name="fastprop_train")

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
FastProp: Trying 331 features... 100% |██████████| [elapsed: 00:01, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:1.177664

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
FastProp: Building features... 100% |██████████| [elapsed: 00:01, remaining: 00:00]          



In [12]:
fastprop_test = pipe_fp_fl.transform(time_series.test, df_name="fastprop_test")

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
FastProp: Building features... 100% |██████████| [elapsed: 00:01, remaining: 00:00]          



Now we create a dedicated prediction pipeline and provide the fast prop features
(contrained in `fastprop_train` and `fastprop_test`.)

In [13]:
predictor = getml.predictors.XGBoostClassifier()

pipe_fp_pr = getml.pipeline.Pipeline(
    tags=["prediction", "fastprop"], predictors=[predictor]
)

In [14]:
pipe_fp_pr.check(fastprop_train)

pipe_fp_pr.fit(fastprop_train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:05, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:5.153885



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostClassifier'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'fastprop'])

In [15]:
pipe_fp_pr.score(fastprop_test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          



,date time,set used,target,accuracy,auc,cross entropy
0,2024-02-21 16:13:26,fastprop_train,Occupancy,0.9995,1.,0.004464
1,2024-02-21 16:13:26,fastprop_test,Occupancy,0.9888,0.9982,0.044271


#### 2.2 Propositionalization with featuretools

In [16]:
data_train = time_series.train.population.to_df("train")
data_test = time_series.test.population.to_df("test")

In [17]:
dfs_pandas = {}

for df in getml.project.data_frames:
    dfs_pandas[df.name] = df.to_pandas()
    dfs_pandas[df.name]["id"] = 1

In [18]:
ft_builder = FTTimeSeriesBuilder(
    num_features=200,
    horizon=pd.Timedelta(minutes=0),
    memory=pd.Timedelta(minutes=15),
    column_id="id",
    time_stamp="date",
    target="Occupancy",
)

The `FTTimeSeriesBuilder` provides a `fit` method that is designed to be equivilant to
to the `fit` method of the predictorless getML pipeline above.

In [19]:
with benchmark("featuretools"):
    featuretools_train = ft_builder.fit(dfs_pandas["train"])

featuretools_test = ft_builder.transform(dfs_pandas["test"])

df_featuretools_train = getml.data.DataFrame.from_pandas(
    featuretools_train, name="featuretools_train", roles=data_train.roles
)
df_featuretools_test = getml.data.DataFrame.from_pandas(
    featuretools_test, name="featuretools_test", roles=data_train.roles
)

df_featuretools_train.set_role(
    df_featuretools_train.roles.unused, getml.data.roles.numerical
)

df_featuretools_test.set_role(
    df_featuretools_test.roles.unused, getml.data.roles.numerical
)

featuretools: Trying features...
Selecting the best out of 262 features...
Time taken: 0h:8m:2.659652



In [20]:
predictor = getml.predictors.XGBoostClassifier()

pipe_ft_pr = getml.pipeline.Pipeline(
    tags=["prediction", "featuretools"], predictors=[predictor]
)

pipe_ft_pr

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostClassifier'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'featuretools'])

In [21]:
pipe_ft_pr.check(df_featuretools_train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

The pipeline check generated 0 issues labeled INFO and 1 issues labeled WARNING.


,type,label,message
0,WARNING,COLUMN SHOULD BE UNUSED,All non-NULL entries in column 'id' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').


In [22]:
pipe_ft_pr.fit(df_featuretools_train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

The pipeline check generated 0 issues labeled INFO and 1 issues labeled WARNING.
To see the issues in full, run .check() on the pipeline.

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:04, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:4.049619



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostClassifier'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'featuretools'])

In [23]:
pipe_ft_pr.score(df_featuretools_test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          



,date time,set used,target,accuracy,auc,cross entropy
0,2024-02-21 16:29:04,featuretools_train,Occupancy,0.9994,1.,0.004997
1,2024-02-21 16:29:04,featuretools_test,Occupancy,0.9886,0.9973,0.05008


#### 2.3 Propositionalization with tsfresh

In [24]:
tsfresh_builder = TSFreshBuilder(
    num_features=200, memory=15, column_id="id", time_stamp="date", target="Occupancy"
)

with benchmark("tsfresh"):
    tsfresh_train = tsfresh_builder.fit(dfs_pandas["train"])

tsfresh_test = tsfresh_builder.transform(dfs_pandas["test"])

df_tsfresh_train = getml.data.DataFrame.from_pandas(
    tsfresh_train, name="tsfresh_train", roles=data_train.roles
)
df_tsfresh_test = getml.data.DataFrame.from_pandas(
    tsfresh_test, name="tsfresh_test", roles=data_train.roles
)

df_tsfresh_train.set_role(df_tsfresh_train.roles.unused, getml.data.roles.numerical)

df_tsfresh_test.set_role(df_tsfresh_test.roles.unused, getml.data.roles.numerical)

Feature Extraction: 100%|██████████| 40/40 [00:06<00:00,  5.80it/s]


Selecting the best out of 65 features...
Time taken: 0h:0m:20.577953



Feature Extraction: 100%|██████████| 40/40 [00:05<00:00,  7.10it/s]


In [25]:
pipe_tsf_pr = getml.pipeline.Pipeline(
    tags=["predicition", "tsfresh"], predictors=[predictor]
)

pipe_tsf_pr

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostClassifier'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['predicition', 'tsfresh'])

In [26]:
pipe_tsf_pr.check(df_tsfresh_train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

The pipeline check generated 0 issues labeled INFO and 1 issues labeled WARNING.


,type,label,message
0,WARNING,COLUMN SHOULD BE UNUSED,All non-NULL entries in column 'id' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').


In [27]:
pipe_tsf_pr.fit(df_tsfresh_train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

The pipeline check generated 0 issues labeled INFO and 1 issues labeled WARNING.
To see the issues in full, run .check() on the pipeline.

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:02, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:1.896389



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostClassifier'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['predicition', 'tsfresh'])

In [28]:
pipe_tsf_pr.score(df_tsfresh_test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          



,date time,set used,target,accuracy,auc,cross entropy
0,2024-02-21 16:29:44,tsfresh_train,Occupancy,0.9985,1.,0.006898
1,2024-02-21 16:29:44,tsfresh_test,Occupancy,0.9877,0.9979,0.049359


### 3. Comparison

In [29]:
num_features = dict(
    fastprop=289,
    featuretools=103,
    tsfresh=60,
)

runtime_per_feature = [
    benchmark.runtimes["fastprop"] / num_features["fastprop"],
    benchmark.runtimes["featuretools"] / num_features["featuretools"],
    benchmark.runtimes["tsfresh"] / num_features["tsfresh"],
]

features_per_second = [1.0 / r.total_seconds() for r in runtime_per_feature]

normalized_runtime_per_feature = [
    r / runtime_per_feature[0] for r in runtime_per_feature
]

comparison = pd.DataFrame(
    dict(
        runtime=[
            benchmark.runtimes["fastprop"],
            benchmark.runtimes["featuretools"],
            benchmark.runtimes["tsfresh"],
        ],
        num_features=num_features.values(),
        features_per_second=features_per_second,
        normalized_runtime=[
            1,
            benchmark.runtimes["featuretools"] / benchmark.runtimes["fastprop"],
            benchmark.runtimes["tsfresh"] / benchmark.runtimes["fastprop"],
        ],
        normalized_runtime_per_feature=normalized_runtime_per_feature,
        accuracy=[pipe_fp_pr.accuracy, pipe_ft_pr.accuracy, pipe_tsf_pr.accuracy],
        auc=[pipe_fp_pr.auc, pipe_ft_pr.auc, pipe_tsf_pr.auc],
        cross_entropy=[
            pipe_fp_pr.cross_entropy,
            pipe_ft_pr.cross_entropy,
            pipe_tsf_pr.cross_entropy,
        ],
    )
)

comparison.index = ["getML: FastProp", "featuretools", "tsfresh"]

In [30]:
comparison

,runtime,num_features,features_per_second,normalized_runtime,normalized_runtime_per_feature,accuracy,auc,cross_entropy
getML: FastProp,0 days 00:00:01.918944,289,150.602410,1.000000,1.000000,0.988823,0.998166,0.044271
featuretools,0 days 00:08:02.660716,103,0.213400,251.524128,705.726807,0.988578,0.997259,0.050080
tsfresh,0 days 00:00:20.578092,60,2.915724,10.723654,51.651807,0.987718,0.997861,0.049359


In [31]:
# export for further use
comparison.to_csv("comparisons/occupancy.csv")